# DSC 80: Project 01

### Checkpoint Due Date: Thursday Jan 16, 11:59:59 PM (Questions 1-4)
### Due Date: Thursday, Jan 23, 11:59:59 PM

---
# Instructions

This Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems.  
* Like the lab, your coding work will be developed in the accompanying `project01.py` file, that will be imported into the current notebook. This code will be autograded.
* **For the checkpoint, turn in questions 1-4**

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are **encouraged to write your own additional functions** to solve the questions! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `project01.py` -- however, be sure to upload these to gradescope as well!
- Always document your code!

**Tips for testing the correctness of your answers!**
Once you have your work saved in the .py file, you should import the `project01` to test your function out in the notebook. In the notebook you should inspect/analyze the output to assess its correctness!
* Run your functions on the main dataset (`grades`) and ask yourself if the output *looks correct.*
* Run your functions on very small datasets (e.g. 1-5 row table), calculate the expected response by hand, and see if the function output matches (this *is* unit-testing your code with data).
* Run your functions on (large and small) samples of the dataset `grades` (with and without replacement). Does your code break? Or does it still run as expected.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import project01 as proj

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime

import os

# The Other Side of Gradescope

The file contains the grade-book from a fictional data science course with 535 students. 

**Note: this dataset is synthetically generated; it does not contain real student grades.**

In this project, you will:
1. clean and process the data to compute total course grades according to a fictional syllabus (below),
2. qualitatively understand how students did in the course,
3. understand how student grades vary with small changes in performance on each assignment.

---

The course syllabus is as follows:

* Lab assignments 
    - Each are worth the same amount, regardless of each lab's raw point total.
    - The lowest lab is dropped.
    - Each lab may be revised for one week after submission for a 10% penalty, for two weeks after submission for a 20% penalty, and beyond that for a 50% penalty. Such revisions are reflected in the `Lateness` columns in the gradebook.
    - Labs are 20% of the total grade.
* Projects 
    - Each project consists of an autograded portion, and *possibly* a free response portion.
    - The total points for a single project consist of the sum of the raw score of the two portions.
    - Each are worth the same amount, regardless of each project's raw point total.
    - Projects are 30% of the total grade.
* Checkpoints
    - Project checkpoints are worth 2.5% of the total grade.
* Discussion
    - Discussion notebooks are worth 2.5% of the total grade.
* Exams
    - The midterm is worth 15% of the total grade.
    - The final is worth 30% of the total grade.


### A note on generalization

You may assume that your code will only need to work on a gradebook for a class with the syllabus given above. That is, you may assume that the dataframe `grades` looks like the given one in `data/grades.csv`.

However, such a class:
1. may have a different numbers of labs, projects, discussions, and project checkpoints.
2. may have a different number of students.

You may assume the course components and the naming conventions are as given in the data file.

The dataset was generated by Gradescope; you must attempt to reason about the data as given using what you know as a student who uses Gradescope.

### A note on 'putting everything together'

The goal of this project is to create and assess final grades for a fictional course; if anything, the process is broken down into functions for your convenience and guidance. Here are a few remarks and tips for approaching the projects:
1. If you are having trouble figuring out what a question is asking you to do, look at the big picture and try to understand what the current step is doing to contribute to this big picture. This may clarify what's being asked!
1. These questions intentionally build off of each other and the final result matters! In fact, you can 'get a question correct', but only receive partial credit on it because a previous answer was wrong.
    - Credit for a question will typically receive partial credit based on *how close* your answer is to correct (as well as some credit for a solution in the correct form). 
    - You should try to assess your answer to each question based on what you understand of the data. This might involve writing extensive code (that isn't turned in) just to check your work! Suggestions on checking your work are given in the assignment, but you should also think of your own ways of checking your work.
    - As you do this project, think about the data from the perspective of the student (which should be easy to do!)

In [4]:
grades_fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(grades_fp)

### Getting started: enumerating the assignments

First, you will list all the 'assignment names' and what part of the syllabus to which they belong.

**Question 1:**

Create a function `get_assignment_names` that takes in a dataframe like `grades` and returns a dictionary with the following structure:
- The keys are the general areas of the syllabus: `lab, project, midterm, final, disc, checkpoint`
- The values are lists that contain the assignment names of that type. For example the lab assignments all have names of the form `labXX` where `XX` is a zero-padded two digit number. See the doctests for more details.

In [5]:
def get_assignment_names(grades):
    from re import search
    dic = {}
    key_list = ['lab', 'project', 'midterm', 'final', 'disc', 'checkpoint']
    rest_column_names = []
    for j in grades.columns:
        if '-' in j:
            rest_column_names.append(j)
    new_colum = grades.copy()
    new_colum = new_colum.drop(columns = rest_column_names)
    for i in key_list:
        dic[i] = []
        for j in new_colum.columns:
            if i in j or i.capitalize() in j:
                if search(i+'[0-9]{2}$',j) :
                    dic[i].append(j)
                elif search(i.capitalize(),j):
                    dic[i].append(j)
                elif search(i+'[a-zA-Z]+[0-9]{2}$',j) :
                    dic[i].append(j)
    return dic

In [6]:
grades_fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(grades_fp)
names = get_assignment_names(grades)
set(names.keys()) == {'lab', 'project', 'midterm', 'final', 'disc', 'checkpoint'}

True

In [7]:
names['final'] == ['Final']

True

In [8]:
'project02' in names['project']

True

In [9]:
'lab01' in names['lab']

True

### Computing project grades

**Question 2**

Compute the total score for the project portion of the course according to the syllabus. Create a function `projects_total` that takes in `grades` and computes the total project grade for the quarter according to the syllabus. The output Series should contain values between 0 and 1.

*Note*: Don't forget to properly handle students who didn't turn in assignments! (Use your experience and common sense).

*Note:* To check your work, try (1) calculating the score for a few types of students by hand, and (2) calculate the statistics for the class performance on each individual course project, making sure they look reasonable.

In [10]:
def projects_total(grades):
    total = 0
    grades = grades.fillna(0.0)
    for col in grades.columns:
        if not "-" in col and "project" in col:
            total += grades[col]
    return total/505

In [11]:
grades_fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(grades_fp)
out = projects_total(grades)
np.all((0 <= out) & (out <= 1))

True

In [12]:
0.7 < out.mean() < 0.9
out.mean()

0.8211492551124273

### Computing lab grades

Now, you will clean and process the lab grades, which is a little more complicated. To do this, you will develop functions that:
- 'normalize' the grades, 
- adjust for late submissions, 
- drop the lowest lab grade, and 
- creates a total lab score for each student.

**Question 3**

Unfortunately, Gradescope sometimes experiences a delay in registering when an assignment is submitted during "periods of heavy usage" (i.e. near a submission deadline). You need to assess when a student's assignment was actually turned in on time, even if Gradescope did not process it in time. To do this, it is helpful to know:
* Every late submission has to be submitted by a TA (late submissions are turned off).
* TAs never submitted a late assignment "just after" the deadline. 
* The deadlines were at midnight and students had to come to staff hours to late-submit their assignment.

Create a function `last_minute_submissions` that takes in the dataframe `grades` and outputs the number of submissions on each assignment that were turned in on time by the student, yet marked 'late' by Gradescope. See the doctest for more details.

*Note:* You have to figure out what truly is a late submission by looking at the data and understanding the facts about the data generating process above. There is some ambiguity in finding which submissions are truly late; you will *make a best guess for a threshold* by looking at this dataset. This question is about 'cleaning' a messy 'data recording process'.

In [13]:
def to_second(td):
    h, m, s = td.split(':')
    seconds = datetime.timedelta(hours = int(h), minutes = int(m), seconds = int(s)).total_seconds()
    return int(seconds)

In [14]:
def last_minute_submissions(grades):
    student_lab_lateness = []
    for col in grades.columns:
        if 'lab' in col and not 'Max' in col and 'Lateness' in col:
            student_lab_lateness.append(col)
    student_lab = grades[student_lab_lateness]
    for col in student_lab:
        student_lab[col] = student_lab[col].astype(str).str.replace(':','').astype(float)
    for col in student_lab:
        student_lab[col] = student_lab[col].between(1,60000)
    new_name = []
    for i in student_lab.columns:
        new_name.append(i[0:5])
    student_lab.columns = new_name
    return student_lab.sum()

In [15]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = last_minute_submissions(grades)
isinstance(out, pd.Series)
out

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


lab01     2
lab02     0
lab03     2
lab04    12
lab05     7
lab06     8
lab07    16
lab08    11
lab09    26
dtype: int64

In [16]:
np.all(out.index == ['lab0%d' % d for d in range(1,10)])

True

In [17]:
(out > 0).sum()

8

**Question 4**

Now you need to adjust the lab grades for late submissions -- however, you need to take into account your investigation in the previous question, since students shouldn't be penalized by a bug in Gradescope!

Create a function `lateness_penalty` that takes in a 'Lateness' column and returns a column of penalties (represented by the values `1.0,0.9,0.8,0.5` according to the syllabus). Only *truly* late submissions should be counted as late.

*Note*: For the purpose of this project, we will only be calculating lateness for labs. There is no penalty for lateness for projects, discussions, nor checkpoints.

In [18]:
def lateness_penalty(col):
    """
    lateness_penalty takes in a 'lateness' column and returns 
    a column of penalties according to the syllabus.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> col = pd.read_csv(fp)['lab01 - Lateness (H:M:S)']
    >>> out = lateness_penalty(col)
    >>> isinstance(out, pd.Series)
    True
    >>> set(out.unique()) <= {1.0, 0.9, 0.8, 0.5}
    True
    """
        
    col = col.apply(lambda x: float(x.replace(':','')))
    def late_val(x):
        if x < 50000:
            return 1.0
        elif x < 1680000 and x > 30000:
            return 0.9
        elif x >= 168000 and x < 3360000:
            return 0.8
        else:
            return 0.5
    col = col.apply(late_val)
    return col

In [19]:
fp = os.path.join('data', 'grades.csv')
col = pd.read_csv(fp)['lab01 - Lateness (H:M:S)']
out = lateness_penalty(col)
out


0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
5      1.0
6      1.0
7      1.0
8      1.0
9      1.0
10     1.0
11     1.0
12     1.0
13     1.0
14     1.0
15     1.0
16     1.0
17     1.0
18     1.0
19     1.0
20     0.9
21     1.0
22     1.0
23     1.0
24     1.0
25     1.0
26     1.0
27     1.0
28     1.0
29     1.0
      ... 
505    1.0
506    0.9
507    1.0
508    1.0
509    1.0
510    1.0
511    1.0
512    1.0
513    1.0
514    0.9
515    1.0
516    1.0
517    1.0
518    1.0
519    1.0
520    1.0
521    1.0
522    1.0
523    1.0
524    1.0
525    1.0
526    1.0
527    1.0
528    1.0
529    1.0
530    0.9
531    1.0
532    1.0
533    1.0
534    1.0
Name: lab01 - Lateness (H:M:S), Length: 535, dtype: float64

In [21]:
set(out.unique()) <= {1.0, 0.9, 0.8, 0.5}
#out.unique()

True

**Question 5**

Create a function `process_labs` that takes in a dataframe like `grades` and returns a dataframe of processed lab scores. The output should:
* share the same index as `grades`,
* have columns given by the lab assignment names (e.g. `lab01,...lab10`)
* have values representing the lab grades for each assignment, adjusted for Lateness and scaled to a score between 0 and 1.

In [24]:


def process_labs(grades):
    new = grades.copy()
    new = new.fillna('0.0')
    names = get_assignment_names(grades)['lab']
    for i in names:
        new[i] = (new[i]/new[i+' - Max Points']) * lateness_penalty(new[i+' - Lateness (H:M:S)'])
    return new[names]

In [25]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = process_labs(grades)
out.columns.tolist() == ['lab%02d' % x for x in range(1,10)]

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

**Question 6**

Create a function `lab_total` that takes in dataframe of processed assignments (like the output of Question 5) and computes the total lab grade for each student according to the syllabus (returning a Series). Your answers should be proportions between 0 and 1. For example, if there are only 3 labs, and a student received scores of {80%,90%,100%}, then the total score would be 0.95.

*Note*: Don't forget to properly handle students who didn't turn in assignments! (Use your experience and common sense).

In [26]:
def lab_total(processed):
    processed = processed.fillna(0)
    result = (processed.sum(axis = 1)-processed.min(axis = 1))/(len(processed.columns)-1)
    return result

In [27]:
cols = 'lab01 lab02 lab03'.split()
processed = pd.DataFrame([[0.2, 0.90, 1.0]], index=[0], columns=cols)
np.isclose(lab_total(processed), 0.95).all()
lab_total(processed)

0    0.95
dtype: float64

### Putting it together

**Question 7**

Finally, you need to create the final course grades. To do this, you will add up the total of each course component according to the weights given in the syllabus. 

* Create a function `total_points` that takes in `grades` and returns the final course grades according to the syllabus. Course grades should be proportions between zero and one.
* Create a function `final_grades` that takes in the final course grades as above and returns a Series of letter grades given by the standard cutoffs (`A >= .90`, `.90 > B >= .80`, `.80 > C >= .70`, `.70 > D >= .60`, `.60 > F`). You should not use rounding to determining the letter grades.
* Create a function `letter_proportions` which takes in the dataframe `grades` and outputs a Series that contains the proportion of the class that received each grade. (This question requires you to put everything together).
* The indices should be ordered by the proportion of the class that receives that grade, from largest to smallest.

*Note 1*: Don't repeat yourself when computing the checkpoint and discussion portions of the course.

*Note 2*: Only the lab portion of the course accounts for late assignments; you may assume all assignments in other portions are turned in without penalty.

*Note 3*: These values should add up to exactly 1.0. If you are getting something close such as 0.99999, that means there is a slight issue with your code from above. 

To check your work, verify the course grade distribution and relevant statistics! Do the work by hand for a few students.

In [28]:
def total_points(grades):
    labs = 0.2
    projects = 0.3
    checkpointp = 0.025
    DI = 0.025
    mt = 0.15 
    final = 0.30
    new = grades.copy()
    new = new.fillna(0)
    project_score = projects_total(grades)
    Lab_score =lab_total(process_labs(grades))
    
    checkpoints = get_assignment_names(grades)['checkpoint']
    discussions = get_assignment_names(grades)['disc']
    for i in checkpoints:
        new[i] = new[i]/new[i+' - Max Points']
    check_score = new[checkpoints].mean(axis = 1)

    for i in discussions:
        new[i] = new[i]/new[i+' - Max Points']
    disc_score = new[discussions].mean(axis = 1)
    
    midterm_scores = new['Midterm']/new['Midterm - Max Points']
    final_exam_scores = new['Final']/new['Final - Max Points']
    final_scores = Lab_score * labs + project_score * projects + check_score * checkpointp + disc_score *DI + mt *midterm_scores + final_exam_scores * final
    return final_scores

In [29]:
def final_grades(total):
    def letter(x):
        if x>=0.9:
            return 'A'
        elif x<0.9 and x>=0.8:
            return 'B'
        elif x<0.8 and x>=0.7:
            return 'C'
        elif x<0.7 and x>=0.6:
            return 'D'
        else:
            return 'F'
    return pd.Series(list(map(letter, total.tolist())))

In [30]:
def letter_proportions(grades):
    dic = {}
    letter_series = final_grades(total_points(grades))
    return letter_series.value_counts()/float(len(grades))

In [31]:
out = final_grades(pd.Series([0.92, 0.81, 0.41]))
out

0    A
1    B
2    F
dtype: object

In [32]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = letter_proportions(grades)
np.all(out.index == ['B', 'C', 'A', 'D', 'F'])
out

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

### Do Sophomores get better grades?

**Question 8**

You notice that students who are sophomores on average did better in the class (if you can't verify this, you should go back and check your work!). Is this difference significant, or just due to noise?

Perform a hypothesis test, assessing likelihood of the null hypothesis: 
> "sophomores earn grades that are roughly equal on average to the rest of the class."


Create a function `simulate_pval` which takes in the number of simulations `N` and `grades` and returns the the likelihood that the grade of sophomores was no better on average than the class as a whole (i.e. calculate the p-value).

*Note:* To check your work, plot the sampling distribution and the observation. Do these values look reasonable?

In [ ]:
def simulate_pval(grades, N):
    grades = grades.fillna(0.0)
    num_soph = grades[grades['Level'] == 'SO'].shape[0]
    class_size = grades.shape[0]
    soph = grades[grades['Level'] == 'SO']
    soph_grade = total_points(soph).mean()
    other_grade = total_points(grades)
    return soph_grade
    result = []
    for i in range(N):
        sample = np.random.choice(other_grade, replace=False, size=num_soph)
        result.append(np.mean(sample))
    return (pd.Series(result) >= soph_grade).mean()

In [ ]:
def simulate_pval(grades, N):
    grades = grades.fillna(0.0)
    num_soph = grades[grades['Level'] == 'SO'].shape[0]
    class_size = grades.shape[0]
    soph = grades[grades['Level'] == 'SO']
    soph_grade = total_points(soph).mean()
    other_grade = total_points(grades)
    output = []
    for i in range(N):
        sample = total_points.sample(num_soph, replace = False)
        output.append(np.mean(sample))
    return np.mean(pd.Series(output >= soph_grade))

In [ ]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = simulate_pval(grades, 100)
0 <= out <= 0.1
out

### What is the true distribution of grades?

The gradebook for this class only reflects one particular instance of each student's performance, subject to the effects of all the little events and hiccups that occurred throughout the quarter. Might you have done better on the midterm had your roommate kept you up all night with their coughing? Wasn't it lucky that the example you were studying just before the final happened to appear on the exam?

**Question 9**

This question will simulate these '(un)lucky, random events' by adding or subtracting random amounts to each assignment before calculating the final grades. These 'random amounts' will be drawn from a Gaussian distribution of mean 0 and a std deviation 0.02:
```
np.random.normal(0, 0.02, size=(num_rows, num_cols))
```
Intuitively, such a model says that random events may bump up or down a given grade (given as a proportion):
- which on average has no effect on the class as a whole (mean 0),
- which not uncommonly might perturb a grade by 2% (std dev 0.02).

Create a function `total_points_with_noise` that takes in a dataframe like `grades`, adds noise to the assignments as described above, and returns the final scores using *the same procedure* as questions 1-7.

*Note:* You should be able to reuse (or minorly change) the code from previous problems. Try to be DRY (don't repeat yourself)!

*Note 1:* Once adding the noise to the assignment scores, use the `np.clip` function to be sure each assignment retains a score between 0% and 100%.

*Note 2:* To check your work -- what would you expect the difference between the actual scores and noisy scores to be, on average?

In [ ]:
def total_points_with_noise(grades):
    labs = 0.2
    projects = 0.3
    checkpointp = 0.025
    DI = 0.025
    mt = 0.15 
    final = 0.30
    data1 = grades.copy()
    data = data1.fillna(0)
    
    lab_names = get_assignment_names(grades)['lab']
    labs_ori = process_labs(data)
    for i in labs_ori.columns:
        data[i] = labs_ori[i]
    
    proj_names = get_assignment_names(grades)['project']
    dic = {}
    for i in proj_names:
        try:
            raw = data[i] + data[i+ '_free_response']
            maxi = data[i + ' - Max Points'] + data[i+ '_free_response - Max Points']
            data[i] = (raw/maxi) 
        except:   
            raw = data[i] 
            maxi = data[i + ' - Max Points']
            data[i] = (raw/maxi)
            
    
    
    checkpoints = get_assignment_names(grades)['checkpoint']
    discussions = get_assignment_names(grades)['disc']
    
    for i in checkpoints:
        data[i] = data[i]/data[i+' - Max Points']


    for i in discussions:
        data[i] = data[i]/data[i+' - Max Points']

    
    midterm_scores = data['Midterm']/data['Midterm - Max Points']
    final_exam_scores = data['Final']/data['Final - Max Points']
    data['Midterm'] = midterm_scores
    data['Final'] = final_exam_scores
    
    selected_col = lab_names + proj_names + checkpoints + discussions + ['Midterm'] +['Final']
    selected_data = data[selected_col]
    manipulate = selected_data + np.random.normal(0, 0.02, size=(selected_data.shape[0], selected_data.shape[1]))
    for i in manipulate.columns:
        manipulate[i] = np.clip(manipulate[i],0,1)
        
    lab_new = lab_total(manipulate[lab_names]) 
    proj_new = manipulate[proj_names].mean(axis = 1)
    check_new = manipulate[checkpoints].mean(axis = 1)
    disc_new = manipulate[discussions].mean(axis = 1)
    midterm_new = manipulate['Midterm']
    final_new = manipulate['Final']
    
    final_scores = lab_new * labs + proj_new * projects + check_new * checkpointp + disc_new *DI + mt *midterm_new+ final_new * final
    return final_scores 

In [ ]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = total_points_with_noise(grades)
np.all((0 <= out) & (out <= 1))
out

### Short-answer questions (hard-coded)

Use your functions from above to understanding the data and answer the following questions. The function below should return **hard-coded values**. It should not compute anything!

**Question 10**

Create a function `short_answer` of zero variables that returns (hard-coded) answers to the following question in a list:
0. For the class on average, what is the difference between students' scores (`total_points`) and their scores with noise (`total_points_with_noise`)? (Remark: plot the distribution of differences; does this align with what you know about binomial distributions?)
1. What percentage of the class only sees their grade change at most (but not including) $\pm 0.01$?
2. What is the 95% confidence interval for the statistic above? (see [DSC10](https://www.inferentialthinking.com/chapters/13/3/Confidence_Intervals.html) and use `np.percentile`)
3. What proportion of the class sees a change in their letter grade?
4. The assumption behind the model in Question 9 is that:
    - The (observed) gradebook well represents the true population of students,
    - The noisy scores represent other possible observations drawn from the true population of students.
    - Answer `True` or `False`

In [ ]:
#0.
grades = pd.read_csv(fp)
out_TP = total_points(grades)
out_TPWN = total_points_with_noise(grades)
out_TP_mean = out_TP.mean()
out_TPWN_mean = out_TPWN.mean()
out_TP_mean == out_TPWN_mean
out_TPWN_mean - out_TP_mean

In [ ]:
#1.


In [ ]:
def short_answer():
    return [0.00069610, 82.878, [79.794392, 85.93925], 0.068897, True]

# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in project01.py

### To submit:
* **Upload the .py file to gradescope**